In [16]:
import numpy as np
import pandas as pd

import dask.dataframe as dd

import string
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
import razdel

from nltk import tokenize as tknz
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from gensim.models import Word2Vec, FastText
import gensim.downloader as api
import annoy

##  Задания
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

--------

О полях датасета:
- дата публикации;
- имя автора;
- текст твита;
- класс, к которому принадлежит текст (положительный, отрицательный, нейтральный);
- количество добавлений сообщения в избранное;
- количество ретвитов (количество копирований этого сообщения другими пользователями);
- количество друзей пользователя;
- количество пользователей, у которых данный юзер в друзьях (количество фоловеров);
- количество листов, в которых состоит пользователь.

In [2]:
columns = ['id',
           'date',
           'author',
           'tweet',
           'class',
           'favorite_count',
           'retweet_cnt',
           'authors_friend_cnt',
           'followers_cnt',
           'groups_cnt'
          ]

In [3]:
df_pos = pd.read_csv('positive.csv', sep=';', names=columns, index_col=False)
df_neg = pd.read_csv('negative.csv', sep=';', names=columns, index_col=False)

C:\Users\WiZor\AppData\Roaming\Python\Python38\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [4]:
df = pd.concat([df_pos, df_neg])
df.reset_index(inplace=True, drop=True)
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,id,date,author,tweet,class,favorite_count,retweet_cnt,authors_friend_cnt,followers_cnt,groups_cnt
0,408906692374446080,2013-12-06 10:32:07,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62
1,408906692693221377,2013-12-06 10:32:07,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59
2,408906695083954177,2013-12-06 10:32:07,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26
3,408906695356973056,2013-12-06 10:32:07,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19
4,408906761416867842,2013-12-06 10:32:23,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  226834 non-null  int64         
 1   date                226834 non-null  datetime64[ns]
 2   author              226834 non-null  object        
 3   tweet               226834 non-null  object        
 4   class               226834 non-null  int64         
 5   favorite_count      226834 non-null  int64         
 6   retweet_cnt         226834 non-null  int64         
 7   authors_friend_cnt  226834 non-null  int64         
 8   followers_cnt       226834 non-null  int64         
 9   groups_cnt          226834 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(2)
memory usage: 17.3+ MB


Класс `TextBuilder` будет реализовывать предварительную обработку текста, применяя ее ко всему датасету с помощью трансляции его к `dask` датасету.

In [6]:
class TextBuilder():
    def __init__(self, df, stop_words=None, text_column='tweet', new_name='prepared_tweet'):
        self._df = df
        self._ddf = dd.from_pandas(df, npartitions=20)
        self._text_column = text_column
        self._new_name = new_name
        self._seg = Segmenter()
        self._emb = NewsEmbedding()
        self._tagger = NewsMorphTagger(self._emb)
        self._vocab = MorphVocab()
        if stop_words is None:
            stop_words = stopwords.words('russian') # + ['rt', '@'] # избыточно при условии len(token.lemma) > 2
        self._stopwords = stop_words
        
    def row_prepare(self, text):
        doc = Doc(text)
        doc.segment(self._seg)
        doc.tag_morph(self._tagger)
        for token in doc.tokens:
            token.lemmatize(self._vocab)
        return [token.lemma for token in doc.tokens if token.pos != 'PUNCT'
                                                        and token.lemma not in self._stopwords
                                                        and len(token.lemma) > 2]
    
    def _apply_to_df(self, df):
        return df.apply(self.row_prepare)
    
    def _dask_apply(self, f):
        return self._ddf[self._text_column].map_partitions(f).compute()

    def prepare(self):
        self._ddf[self._new_name] = self._dask_apply(self._apply_to_df)
        self._df = self._ddf.compute()
        return self._df
    
    def df(self):
        return self._df
        

In [8]:
%%time
builder = TextBuilder(df)
builder.prepare().head()

Wall time: 12min 16s


,id,date,author,tweet,class,favorite_count,retweet_cnt,authors_friend_cnt,followers_cnt,groups_cnt,prepared_tweet
0,408906692374446080,2013-12-06 10:32:07,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,"[first_timee, школоть, поверь, самый, общество..."
1,408906692693221377,2013-12-06 10:32:07,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,"[все-таки, немного, похожий, мальчик, весь, ра..."
2,408906695083954177,2013-12-06 10:32:07,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,"[katiacheh, идиотка, испугаться]"
3,408906695356973056,2013-12-06 10:32:07,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,"[digger, 2912, угол, сидеть, погибать, голод, ..."
4,408906761416867842,2013-12-06 10:32:23,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,"[irina_dyshkant, значить, страшилка, блин, пос..."


In [9]:
sentences = builder.df()['prepared_tweet']
sentences

0         [first_timee, школоть, поверь, самый, общество...
1         [все-таки, немного, похожий, мальчик, весь, ра...
2                          [katiacheh, идиотка, испугаться]
3         [digger, 2912, угол, сидеть, погибать, голод, ...
4         [irina_dyshkant, значить, страшилка, блин, пос...
                                ...                        
226829        [каждый, хотеть, исправлять, http, qnoddqzuz]
226830    [скучать, taaannyaaa, вправляет, мозг, весь, р...
226831                            [школа, говно, это, идти]
226832         [them, lisaberoud, тауриэль, грусть, обнять]
226833    [такси, везти, работа, раздумывать, приплатить...
Name: prepared_tweet, Length: 226834, dtype: object

Оценим адекватность полученных самых популярных слов:

In [10]:
res = []
w_counts = dict()
for d in sentences:
    res += d

for w in res:
    w_counts[w] = w_counts.get(w, 0) + 1

# w_counts = sorted(w_counts, key=lambda x:)
{k:v for k, v in sorted(w_counts.items(), key=lambda x: x[1], reverse=True)[:20]}

{'http': 33072,
 'весь': 29865,
 'это': 22598,
 'хотеть': 11790,
 'день': 11086,
 'мочь': 9159,
 'сегодня': 8786,
 'очень': 7484,
 'знать': 6820,
 'год': 6796,
 'хороший': 6776,
 'просто': 6534,
 'человек': 5980,
 'любить': 5840,
 'свой': 5681,
 'завтра': 5346,
 'новый': 5316,
 'вообще': 4780,
 'делать': 4546,
 'спасибо': 4384}

Кроме `http` никаких подозрительных вариантов нет, но и его можно оставить, скорее это часть ссылок, которые были распарсены на составные части. По хорошему, если убирать этот токен, то необходимо убирать все его составляющие перед этапом токенизации (например, при нахождении `http` убрать все до следующего пробела, т.к. в самой ссылке обычно их нет). Но на текущем этапе этим заниматься не будем.

In [11]:
dim = 200

modelW2V = Word2Vec(sentences=builder.df()['prepared_tweet'], vector_size=dim, window=5, min_count=1)
modelFT = FastText(sentences=builder.df()['prepared_tweet'], vector_size=dim, window=5, min_count=1)

# word_vectors = api.load('glove-twitter-100')

In [72]:
builder.df().head(5)

,id,date,author,tweet,class,favorite_count,retweet_cnt,authors_friend_cnt,followers_cnt,groups_cnt,prepared_tweet
0,408906692374446080,2013-12-06 10:32:07,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,"[first_timee, школоть, поверь, самый, общество..."
1,408906692693221377,2013-12-06 10:32:07,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,"[все-таки, немного, похожий, мальчик, весь, ра..."
2,408906695083954177,2013-12-06 10:32:07,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,"[katiacheh, идиотка, испугаться]"
3,408906695356973056,2013-12-06 10:32:07,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,"[digger, 2912, угол, сидеть, погибать, голод, ..."
4,408906761416867842,2013-12-06 10:32:23,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,"[irina_dyshkant, значить, страшилка, блин, пос..."


In [52]:
def fill_index(series_tokens, series_origin, dim, n_clus):
    
    assert len(series_tokens) == len(series_origin), '"series_tokens" и "series_origin" должны быть одинаковой размерности'
    w2v_index = annoy.AnnoyIndex(dim ,'angular')
    ft_index = annoy.AnnoyIndex(dim ,'angular')

    mapping = {}
    counter = 0

    for i, line in enumerate(series_tokens):
        n_w2v = 0
        n_ft = 0
        
        mapping[counter] = series_origin[i]
        vector_w2v = np.zeros(dim)
        vector_ft = np.zeros(dim)
        for word in line:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

    w2v_index.build(n_clus)
    ft_index.build(n_clus)
    return mapping, w2v_index, ft_index

In [73]:
def get_response(text, preprocessor, model, index, n_closest, mapping, dim):
    tokens = preprocessor(text)
    vector = np.zeros(dim)
    norm = 0
    for t in tokens:
        if t in model.wv:
            vector += model.wv[t]
            norm += 1
    if norm > 0:
        vector = vector / norm
    closests = index.get_nns_by_vector(vector, n_closest)
    return [mapping[n] for n in closests]

Инициализируем поля-индексы для каждого метода и проверим результат:

In [ ]:
mapping, w2v_index, ft_index = fill_index(builder.df()['prepared_tweet'], builder.df()['tweet'], dim, 10)

In [74]:
text = 'как спалось?'

In [75]:
get_response(text, builder.row_prepare, modelW2V, w2v_index, 5, mapping, dim)

['Кумарят эти "прик", "нрав", "кул", "найс"....тошнит!! ((',
 'Кумарят эти "прик", "нрав", "кул", "найс"....тошнит!! ((',
 '@imluluuu окей с:\nУже лучше, даже насморка почти нет х)',
 '@Ilya_Chernyak Нежности еще ему захотелось. Затрахаю до крови , а потом перережу горло)',
 'Чертов сонный паралич и живот. Съела опять аптеку и не помогает:(']

In [76]:
get_response(text, builder.row_prepare, modelFT, ft_index, 5, mapping, dim)

['Ну, да начнётся ночь, надеюсь она не будет очень скучной и надеюсь мне что-нибудь присниться :) хоть я и спать не собираюсь, но подутроустну',
 'Надо бы наверно и спать уже ложиться — совсем график сна сбился :)',
 'Снились сегодня ужасные сны. Никак не выкину Его из головы. Снился Он и Она. Проснулась в слезах :(',
 'RT @kitty_died: Ну каааааак я умудряюсь даже во сне о тебе думать\nАж снился\nВстала и опять ты :((((',
 'У меня на сон осталось всего 3ч 30м ( да и спать особо не хочется...']

На первый взгляд метод `Fast Text` справляется куда лучше, однако не было проверено других комбинация при обучении, но даже со стандартными результат виден сразу.